In [1]:
function mult2_seq(xs)
    ys = ()
    for x in xs
        ys = (ys..., x * 2)
    end
    return ys
end

mult2_seq (generic function with 1 method)

In [2]:
mult2_seq((1, 2, 3))

(2, 4, 6)

In [3]:
function dfa_parity(xs)
    ys = ()
    state = :s0
    for x in xs
        if state == :s0
            if x == 0
                state = :s0
                ys = (ys..., :even)
            elseif x == 1
                state = :s1
                ys = (ys..., :odd)
            else
                error("Unexpected input: ", x)
            end
        elseif state == :s1
            if x == 0
                state = :s1
                ys = (ys..., :odd)
            elseif x == 1
                state = :s0
                ys = (ys..., :even)
            else
                error("Unexpected input: ", x)
            end
        end
    end
    return ys
end

dfa_parity (generic function with 1 method)

In [4]:
dfa_parity((0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [5]:
function dfa_run(tr, start, xs)
    ys = ()
    state = start
    for x in xs
        (s, y) = tr(state, x)
        state = s
        ys = (ys..., y)
    end
    return ys
end

dfa_run (generic function with 1 method)

In [6]:
function tr_parity1(state, x)
    if state == :s0
        if x == 0
            :s0, :even
        elseif x == 1
            :s1, :odd
        else
            error("Unexpected input: ", x)
        end
    elseif state == :s1
        if x == 0
            :s1, :odd
        elseif x == 1
            :s0, :even
        else
            error("Unexpected input: ", x)
        end
    end
end

tr_parity1 (generic function with 1 method)

In [7]:
dfa_run(tr_parity1, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [8]:
parity_table = (
    (:s0, (
        (0, (:s0, :even)),
        (1, (:s1, :odd)))),
    (:s1, (
        (0, (:s1, :odd)),
        (1, (:s0, :even))))
)

((:s0, ((0, (:s0, :even)), (1, (:s1, :odd)))), (:s1, ((0, (:s1, :odd)), (1, (:s0, :even)))))

In [9]:
function tr_select(tbl, state, input)
    for (s, r) in tbl
        if s == state
            for (x, t) in r
                if x == input
                    return t
                end
            end
            error("Unknown input: ", input)        
        end
    end
    error("Unknown state: ", state)
end

tr_select (generic function with 1 method)

In [10]:
tr_select(parity_table, :s1, 1)

(:s0, :even)

In [11]:
function dfa_run_tbl(tbl, start, xs)
    ys = ()
    state = start
    for input in xs
        (s, y) = tr_select(tbl, state, input)
        state = s
        ys = (ys..., y)
    end
    return ys
end

dfa_run_tbl (generic function with 1 method)

In [12]:
dfa_run_tbl(parity_table, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [13]:
using CompTime

In [14]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [15]:
@ct_enable function tr_select_ct(@ct(tbl), @ct(state), input)
    @ct_ctrl for (s, r) in tbl
        @ct_ctrl if s == state
            @ct_ctrl for (x, t) in r
                if @ct(x) == input
                    return @ct(t)
                end
            end
            error("Unknown input: ", input)        
        end
    end
    error("Unknown state: ", state)
end

runtime (generic function with 1 method)

In [16]:
debug(tr_select_ct, Val{parity_table}, Val{:s1}, 1) |> cleanup

quote
    if 0 == input
        return (:s1, :odd)
    end
    if 1 == input
        return (:s0, :even)
    end
    error("Unknown input: ", input)
    error("Unknown state: ", state)
end

In [17]:
tr_select_ct(Val{parity_table}, Val{:s1}, 1)

(:s0, :even)

In [30]:
@ct_enable function dfa_run_tbl_ct(@ct(tbl), @ct(start), xs)
    ys = ()
    @ct state = start
    for input in xs
        (s, y) = tr_select(@ct(tbl), @ct(state), input)
        @ct state = s
        ys = (ys..., y)
    end
    return ys
end

runtime (generic function with 2 methods)

In [31]:
dfa_run_tbl_ct(Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0))

UndefVarError: UndefVarError: `s` not defined

In [32]:
debug(dfa_run_tbl_ct, Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0)) |> cleanup

UndefVarError: UndefVarError: `s` not defined